In [58]:
import pandas as pd
from numpy import mean
from numpy import std
from numpy import absolute
from matplotlib import pyplot
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import TransformedTargetRegressor

In [59]:
dataset = pd.read_csv('Hasil Tes SBK LPDP Gelombang I - 2021 (Responses) - Form Responses 1.csv')
dataset

,Timestamp,Jenis Beasiswa,Tingkat Pendidikan yang Dituju,Negara Tujuan Kuliah,Bidang Studi yang dipilih saat mendaftar,Mendaftar dengan menggunakan LoA sesuai ketentuan LPDP,Apakah anda dinyatakan Lulus Seleksi Substansi Akademik Pada gelombang I Tahun 2021?,Jumlah Jawaban Benar pada Tes Penalaran Verbal,Jumlah Jawaban Benar pada Tes Kuantitatif,Jumlah Jawaban Benar pada Tes Pemecahan Masalah,[optional] Kampus/universitas tujuan saat ini (pilihan 1)
0,6/24/2021 15:54:15,Targeted,S2,Dalam negeri,TRP,Tidak,Ya,10,12,6,Universitas Indonesia
1,6/24/2021 15:54:22,Reguler,S2,Luar Negeri,ENE,"Ya, dengan Loa",Ya,17,22,8,UCL
2,6/24/2021 15:54:54,Targeted,S2,Luar Negeri,ELE,"Ya, dengan Loa",Ya,16,18,7,KTH Sweden
3,6/24/2021 15:56:06,Reguler,S2,Dalam negeri,LIN,Tidak,Tidak,15,12,4,UGM
4,6/24/2021 15:56:28,Afirmasi,S2,Luar Negeri,LIN,Tidak,Ya,16,9,7,University of Birmingham
...,...,...,...,...,...,...,...,...,...,...,...
426,10/13/2021 20:21:59,Reguler,S2,Dalam negeri,718,Tidak,Tidak,23,25,12,UGM
427,10/16/2021 8:54:13,Reguler,S2,Dalam negeri,TRP,Tidak,Tidak,14,14,6,UI
428,10/18/2021 14:20:18,Targeted,S2,Luar Negeri,ENT,Tidak,Tidak,15,7,9,Imperial College London
429,10/19/2021 21:23:43,Targeted,S2,Dalam negeri,TRP,Tidak,Ya,17,11,6,NaN


In [60]:
new_dataset = dataset.drop(['Timestamp', '[optional] Kampus/universitas tujuan saat ini (pilihan 1)'], axis = 1)
new_dataset

,Jenis Beasiswa,Tingkat Pendidikan yang Dituju,Negara Tujuan Kuliah,Bidang Studi yang dipilih saat mendaftar,Mendaftar dengan menggunakan LoA sesuai ketentuan LPDP,Apakah anda dinyatakan Lulus Seleksi Substansi Akademik Pada gelombang I Tahun 2021?,Jumlah Jawaban Benar pada Tes Penalaran Verbal,Jumlah Jawaban Benar pada Tes Kuantitatif,Jumlah Jawaban Benar pada Tes Pemecahan Masalah
0,Targeted,S2,Dalam negeri,TRP,Tidak,Ya,10,12,6
1,Reguler,S2,Luar Negeri,ENE,"Ya, dengan Loa",Ya,17,22,8
2,Targeted,S2,Luar Negeri,ELE,"Ya, dengan Loa",Ya,16,18,7
3,Reguler,S2,Dalam negeri,LIN,Tidak,Tidak,15,12,4
4,Afirmasi,S2,Luar Negeri,LIN,Tidak,Ya,16,9,7
...,...,...,...,...,...,...,...,...,...
426,Reguler,S2,Dalam negeri,718,Tidak,Tidak,23,25,12
427,Reguler,S2,Dalam negeri,TRP,Tidak,Tidak,14,14,6
428,Targeted,S2,Luar Negeri,ENT,Tidak,Tidak,15,7,9
429,Targeted,S2,Dalam negeri,TRP,Tidak,Ya,17,11,6


In [61]:
# 10.2 sebelum melakukan transformasi
X = new_dataset.iloc[:, [0,1,2,3,4,6,7,8]]
y = new_dataset.iloc[:, 5] 

In [62]:
numerical = X.select_dtypes(include=['int64']).columns
categorical = X.select_dtypes(include=['object']).columns

In [63]:
le = LabelEncoder()
y_le = le.fit_transform(y)

In [64]:
col_trans = [('or', OrdinalEncoder(), categorical)]
column_transformer = ColumnTransformer(transformers = col_trans)
X_oe = column_transformer.fit_transform(X)

In [65]:
model = TransformedTargetRegressor(regressor=LogisticRegression())
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, X_oe, y_le, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores_acc = cross_val_score(model, X_oe, y_le, scoring='accuracy', cv=cv, n_jobs=-1)

scores = absolute(scores)
s_mean = mean(scores)
s_mean_acc = mean(scores_acc)
print('Mean MAE: %.3f' % (s_mean))
print('Accuracy: %.3f' % (s_mean_acc))
print('\n')

Mean MAE without transform: 0.210
Accuracy without transform: 0.790




In [66]:
# 10.2 setelah melakukan transformasi
col_trans_1 = [('or', OrdinalEncoder(), categorical),('tr', PowerTransformer(method='yeo-johnson'), numerical)]
column_transformer = ColumnTransformer(transformers = col_trans_1)
X_tr = column_transformer.fit_transform(X)

In [67]:
model = TransformedTargetRegressor(regressor=LogisticRegression())
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, X_tr, y_le, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores_acc = cross_val_score(model, X_tr, y_le, scoring='accuracy', cv=cv, n_jobs=-1)

scores = absolute(scores)
s_mean = mean(scores)
s_mean_acc = mean(scores_acc)
print('Mean MAE: %.3f' % (s_mean))
print('Accuracy: %.3f' % (s_mean_acc))
print('\n')

Mean MAE: 0.197
Accuracy: 0.803


